In [1]:
import tensorflow as tf 
from tensorflow import keras as keras
from tensorflow.python.keras import backend as K
import os
import numpy as np
import datetime
import importlib


import Clone_Model
import rsc

In [2]:
checkpoint_path = "./cloning/checkpoints/cloning/default_checkpoint.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
ckpt_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, save_weights_only=True, verbose=1)

log_dir_clone = "./cloning/logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir_clone, histogram_freq=1)

In [39]:
#      CLONE MODEL EAGER ENABLED
###########################################

importlib.reload(rsc)
importlib.reload(Clone_Model)

if not tf.executing_eagerly(): tf.compat.v1.enable_eager_execution()

K.clear_session()

batch_size = 3

train, test = rsc.get_titanic_dataset()


model = rsc.get_model_slim()

model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.BinaryCrossentropy(),
    run_eagerly = tf.executing_eagerly()
    )


history = model.fit(train,
                    batch_size=batch_size,
                    epochs=1, 
                    steps_per_epoch=3, 
                    validation_data=test, 
                    validation_steps=1, 
                    callbacks=[tensorboard_callback])


clone = Clone_Model.satify_model(model)

clone.compile(optimizer=keras.optimizers.Adam(),
              loss=keras.losses.BinaryCrossentropy(),
              run_eagerly = tf.executing_eagerly())

sat_cb = Clone_Model.sat_results()

history = clone.fit(train,
                    batch_size=batch_size,
                    epochs = 1,
                    steps_per_epoch=3,
                    validation_data=test, 
                    validation_steps=1, 
                    callbacks=[tensorboard_callback, sat_cb])
                

3/3 [==============================] - 0s 12ms/step - loss: 0.6941 - val_loss: 0.6961
Init Satlayer!
Init Satlayer!
Init Satlayer!
Init Satlayer!
Init Satlayer!
SatLayer.update(): Input tensor has shape: (None, 3)
SatLayer.update(): Input tensor has shape: (None, 3)
SatLayer.update(): Input tensor has shape: (None, 2)
SatLayer.update(): Input tensor has shape: (None, 2)
SatLayer.update(): Input tensor has shape: (None, 2)
SatLayer.update(): Input tensor has shape: (None, 3)
SatLayer.update(): Input tensor has shape: (None, 3)
SatLayer.update(): Input tensor has shape: (None, 2)
SatLayer.update(): Input tensor has shape: (None, 2)
SatLayer.update(): Input tensor has shape: (None, 2)
Clone_Model.satify_model() end.
SatLayer.update(): Input tensor has shape: (20, 3)
SatLayer.update(): Input tensor has shape: (20, 3)
SatLayer.update(): Input tensor has shape: (20, 2)
SatLayer.update(): Input tensor has shape: (20, 2)
SatLayer.update(): Input tensor has shape: (20, 2)
1/3 [=========>.......

In [114]:
#      TRAIN MODEL EAGER ENABLED MNIST CLASSIFICATION
###########################################

importlib.reload(rsc)
importlib.reload(Clone_Model)

if not tf.executing_eagerly(): tf.compat.v1.enable_eager_execution()

K.clear_session()

batch_size = 20

train, test = rsc.get_mnist()


model = rsc.get_model_unitlist(hidden_layers_spec=[128])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])



history = model.fit(train[0],
                    train[1],
                    batch_size=batch_size,
                    epochs=10, 
                    steps_per_epoch=None, 
                    validation_data=test, 
                    validation_steps=1, 
                    callbacks=[tensorboard_callback])



Epoch 1/10
3000/3000 [==============================] - 12s 4ms/step - loss: 2.2026 - accuracy: 0.8638 - val_loss: 0.7533 - val_accuracy: 0.9000
Epoch 2/10
3000/3000 [==============================] - 10s 3ms/step - loss: 0.3796 - accuracy: 0.9107 - val_loss: 0.2871 - val_accuracy: 0.9000
Epoch 3/10
3000/3000 [==============================] - 9s 3ms/step - loss: 0.2980 - accuracy: 0.9265 - val_loss: 0.2013 - val_accuracy: 0.9000
Epoch 4/10
3000/3000 [==============================] - 9s 3ms/step - loss: 0.2702 - accuracy: 0.9352 - val_loss: 0.2948 - val_accuracy: 0.9500
Epoch 5/10
3000/3000 [==============================] - 8s 3ms/step - loss: 0.2509 - accuracy: 0.9410 - val_loss: 0.3036 - val_accuracy: 0.9000
Epoch 6/10
3000/3000 [==============================] - 7s 2ms/step - loss: 0.2457 - accuracy: 0.9428 - val_loss: 0.1963 - val_accuracy: 0.9000
Epoch 7/10
3000/3000 [==============================] - 10s 3ms/step - loss: 0.2393 - accuracy: 0.9457 - val_loss: 0.2068 - val_accura

In [246]:
#      CLONE MODEL EAGER ENABLED MNIST CLASSIFICATION
###########################################

importlib.reload(rsc)
importlib.reload(Clone_Model)


clone = Clone_Model.satify_model(model)

"""clone.compile(optimizer=keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              run_eagerly = tf.executing_eagerly())
"""
sat_cb = Clone_Model.sat_results()

history = clone.fit(train[0],
                    train[1],
                    batch_size=10,
                    epochs = 1,
                    steps_per_epoch=5,
                    validation_data=test, 
                    validation_steps=1, 
                    callbacks=[sat_cb])
              

<class 'tensorflow.python.keras.layers.core.Flatten'>
ListWrapper(['o_s', 'r_s', 's_s'])
ListWrapper(['o_s', 'r_s', 's_s'])
RESET Weights!
RESET Weights!
ACT TENSOR DENSE.CALL: Tensor("sequential/dense/Relu:0", shape=(None, 128), dtype=float32)
SATLAYER UPDATE: Tensor("sequential/dense/Cast:0", shape=(None, 128), dtype=float64)
AGGREG UPDATESTATE() CALLED: 0.0
AGGREG UPDATESTATE() CALLED: Tensor("sequential/dense/Sum_1:0", shape=(128,), dtype=float64)
AGGREG UPDATESTATE() CALLED: Tensor("sequential/dense/MatMul_2:0", shape=(128, 128), dtype=float64)
ACT TENSOR DENSE.CALL: Tensor("sequential/dense_1/BiasAdd:0", shape=(None, 10), dtype=float32)
SATLAYER UPDATE: Tensor("sequential/dense_1/Cast:0", shape=(None, 10), dtype=float64)
AGGREG UPDATESTATE() CALLED: 0.0
AGGREG UPDATESTATE() CALLED: Tensor("sequential/dense_1/Sum_1:0", shape=(10,), dtype=float64)
AGGREG UPDATESTATE() CALLED: Tensor("sequential/dense_1/MatMul_2:0", shape=(10, 10), dtype=float64)
Clone_Model.satify_model() end.
ACT 

1/5 [=====>........................] - ETA: 0s - loss: 3.2429 - accuracy: 1.0000ACT TENSOR DENSE.CALL: [[  0.        0.        0.      ...   0.      298.90533   0.     ]
 [  0.        0.        0.      ...   0.        0.        0.     ]
 [  0.        0.        0.      ...   0.        0.        0.     ]
 ...
 [  0.        0.        0.      ...   0.        0.        0.     ]
 [  0.        0.        0.      ...   0.      439.973     0.     ]
 [  0.        0.      544.93726 ...   0.      252.1985    0.     ]]
SATLAYER UPDATE: [[  0.           0.           0.         ...   0.         298.90533447
    0.        ]
 [  0.           0.           0.         ...   0.           0.
    0.        ]
 [  0.           0.           0.         ...   0.           0.
    0.        ]
 ...
 [  0.           0.           0.         ...   0.           0.
    0.        ]
 [  0.           0.           0.         ...   0.         439.97299194
    0.        ]
 [  0.           0.         544.93725586 ...   0.       

SATLAYER UPDATE: [[  0.           0.           0.         ...   0.           0.
    0.        ]
 [  0.           0.           0.         ...   0.           0.
    0.        ]
 [  0.           0.           0.         ...   0.           0.
    0.        ]
 ...
 [  0.           0.           0.         ...   0.           0.
    0.        ]
 [  0.           0.         702.79211426 ...   0.          71.73034668
    0.        ]
 [  0.           0.           0.         ...   0.           0.
    0.        ]]
AGGREG UPDATESTATE() CALLED: 10
AGGREG UPDATESTATE() CALLED: [   0.            0.         1104.52401733    0.            0.
    0.            0.          315.48800659    0.            0.
  538.77807617    0.            0.            0.            0.
    0.            0.            0.            0.            0.
    0.            0.          649.40667725    0.            0.
    0.            0.            0.            0.            0.
  203.38735962    0.            0.            0.        

SATLAYER UPDATE: [[  0.           0.           0.         ...   0.           0.
    0.        ]
 [  0.           0.           0.         ...   0.           0.
    0.        ]
 [  0.           0.           0.         ...   0.           0.
    0.        ]
 ...
 [  0.           0.           0.         ...   0.           0.
    0.        ]
 [  0.           0.           0.         ...   0.           0.
    0.        ]
 [  0.           0.         701.20233154 ...   0.         535.38494873
    0.        ]]
AGGREG UPDATESTATE() CALLED: 10
AGGREG UPDATESTATE() CALLED: [   0.            0.          701.20233154    0.            0.
    0.            0.          403.30722046    0.            0.
  807.24438477    0.            0.            0.            0.
    0.            0.          552.3515625     0.            0.
  137.00296021    0.          813.90002441    0.            0.
    0.          104.7791748     0.            0.            0.
    0.            0.            0.            0.        

SATLAYER UPDATE: [[ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          4.67937183 ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 ...
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.         45.82513428
   0.        ]]
AGGREG UPDATESTATE() CALLED: 10
AGGREG UPDATESTATE() CALLED: [   0.            0.            4.67937183    0.            0.
    0.            0.          384.23003006    0.            0.
  406.61941528  313.49523926    0.            0.            0.
    0.            0.            0.            0.            0.
    0.            0.           22.92939186    0.            0.
    0.            0.            0.            0.            0.
 1031.79553223    0.            0.            0.            0.
    0.            0.         

AGGREG UPDATESTATE() CALLED: 10
AGGREG UPDATESTATE() CALLED: [   0.            0.          696.00213623    0.            0.
    0.            0.            0.            0.            0.
  795.15887451  713.87103271    0.            0.            0.
    0.            0.          294.69897461    0.            0.
    0.            0.          392.58843994    0.            0.
    0.          186.57922745    0.            0.            0.
   44.48922729    0.            0.            0.            0.
    0.            0.            0.            0.            0.
    0.            0.            0.            0.            0.
  565.66628933    0.            0.            0.            0.
    0.            0.            0.            0.            0.
    0.            0.            0.            0.            0.
  372.07038116    0.            0.            0.            0.
    0.            0.          357.28430176    0.            0.
    0.          428.81161499  582.15634155  238.90670776 

In [62]:
in_shape = model.layers[0].input_shape[-2:]
print(in_shape)
np.ones(shape=in_shape).shape

(28, 28)


(28, 28)

In [46]:
#(None, 21, 5)[1:]*2
#np.zeros(shape=(None))
importlib.reload(rsc)
train_b, test_b = rsc.get_titanic_dataset(batch_size=3)
print(train_b)
print(model(np.array([[1,2,3,4] ])))

for ix, x in enumerate(train_b):
    print(x)

<BatchDataset shapes: ((None, None, 4), (None, None, 2)), types: (tf.float32, tf.int32)>
tf.Tensor([[0.69211704 0.3078829 ]], shape=(1, 2), dtype=float32)
(<tf.Tensor: shape=(3, 20, 4), dtype=float32, numpy=
array([[[ 61.    ,   0.    ,   0.    ,  32.3208],
        [ 38.    ,   0.    ,   1.    , 153.4625],
        [ 36.    ,   0.    ,   0.    ,  26.3875],
        [ 28.    ,   0.    ,   0.    ,  26.55  ],
        [ 22.    ,   0.    ,   0.    ,  10.5167],
        [ 28.    ,   3.    ,   1.    ,  25.4667],
        [ 28.    ,   0.    ,   0.    ,   7.8958],
        [ 30.    ,   0.    ,   0.    ,   7.8958],
        [ 34.    ,   1.    ,   0.    ,  21.    ],
        [  3.    ,   4.    ,   2.    ,  31.3875],
        [ 31.    ,   1.    ,   0.    ,  57.    ],
        [ 16.    ,   0.    ,   0.    ,   8.05  ],
        [ 28.    ,   3.    ,   1.    ,  25.4667],
        [ 39.    ,   0.    ,   0.    ,   0.    ],
        [ 28.    ,   0.    ,   0.    , 227.525 ],
        [ 19.    ,   0.    ,   0.    ,   7

InvalidArgumentError: Cannot add tensor to the batch: number of elements does not match. Shapes are: [tensor]: [7,4], [batch]: [20,4]